In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary

In [2]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_channels:int=3, patch_size:int=16, emb_size:int=768, img_size:int=224):
        self.P = patch_size
        super().__init__()
        self.projection = nn.Sequential(
            nn.Conv2d(in_channels,emb_size, kernel_size=patch_size, stride=patch_size),
            Rearrange('b e h w -> b (h w) e')
        )
        self.cls_token= nn.Parameter(torch.randn(1,1,emb_size))
        self.positions = nn.Parameter(torch.randn((img_size//patch_size)**2+1, emb_size))
    def forward(self, x:Tensor)->Tensor:
        b,_,_,_ = x.shape
        x =self.projection(x)
        cls_tokens = repeat(self.cls_token, '() n e -> b n e', b=b)
        x = torch.cat([cls_tokens, x], dim=1)
        x+= self.positions
        return x

In [3]:
class MultiHeadAttention(nn.Module) :
    def __init__(self, emb_size:int=768, num_heads:int=8, dropout:float=0):
        super().__init__()
        self.emb_size= emb_size
        self.num_heads = num_heads
        #Fuse the queries, keys, values in one matrix
        self.qkv = nn.Linear(emb_size, emb_size*3)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)
        
    def forward(self, x:Tensor, mask:Tensor = None)-> Tensor:
        #split keys, queries, and vlaues in num_heads
        qkv = rearrange(self.qkv(x), "b n (h d qkv) -> (qkv) b h n d", h = self.num_heads, qkv=3)
        queries, keys, values = qkv[0], qkv[1], qkv[2]
        
        #sum up over the last axis
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys) #batch, num_heads, query_len, key_len
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)
        
        scaling = self.emb_size**(1/2)
        att = F.softmax(energy, dim =-1) / scaling
        att = self.att_drop(att)
        
        # sum up over the third axis
        out = torch.einsum('bhal, bhlv -> bhav', att, values)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out = self.projection(out)
        return out

In [4]:
class ResidualAdd(nn.Module):
    def __init__(self,fn):
        super().__init__()
        self.fn = fn
    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x+= res
        return x

In [5]:
class FeedForwardBlock(nn.Sequential):
    def __init__(self,emb_size:int, expansion : int=4, drop_p :float = 0.):
        super().__init__(
            nn.Linear(emb_size, expansion* emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion*emb_size, emb_size))

In [6]:
class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                emb_size:int=768,
                drop_p : float=0,
                forward_expansion : int=4,
                forward_drop_p : float=0.,
                **kwargs):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, **kwargs),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion,
                    drop_p = forward_drop_p),
                )
            ))

In [7]:
class TransformerEncoder(nn.Sequential):
    def __init__(self, depth:int =12, **kwargs):
        super().__init__(*[TransformerEncoderBlock(**kwargs) for _ in range(depth)])

In [8]:
class ClassificationHead(nn.Sequential):
    def __init__(self, emb_size:int =768, n_classes: int=10):
        super().__init__(
            Reduce('b n e -> b e', reduction ='mean'),
            nn.LayerNorm(emb_size),
            nn.Linear(emb_size, n_classes))

In [9]:
class ViT(nn.Sequential):
    def __init__(self, 
                in_channels: int=3,
                patch_size : int=16,
                emb_size :int=768,
                img_size:int=12,
                depth: int=12,
                n_classes:int =10,
                **kwargs):
        super().__init__(
            PatchEmbedding(in_channels, patch_size, emb_size, img_size),
        TransformerEncoder(depth, emb_size=emb_size, **kwargs),
        ClassificationHead(emb_size, n_classes))

In [11]:
net = ViT()
net.to('cuda')

summary(net, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
         Rearrange-2             [-1, 196, 768]               0
    PatchEmbedding-3             [-1, 197, 768]               0
         LayerNorm-4             [-1, 197, 768]           1,536
            Linear-5            [-1, 197, 2304]       1,771,776
           Dropout-6          [-1, 8, 197, 197]               0
            Linear-7             [-1, 197, 768]         590,592
MultiHeadAttention-8             [-1, 197, 768]               0
           Dropout-9             [-1, 197, 768]               0
      ResidualAdd-10             [-1, 197, 768]               0
        LayerNorm-11             [-1, 197, 768]           1,536
           Linear-12            [-1, 197, 3072]       2,362,368
             GELU-13            [-1, 197, 3072]               0
          Dropout-14            [-1, 19